# GEMINI 2.5 Flash
## Refined mutation operator generated by Gemini 2.5 Flash during Stage 2 of the proposed refinement framework.

In [1]:
# imports 
import os
import numpy as np
import ioh 
import pandas as pd
import inspect
import importlib
import re 

In [2]:
# First, we define the DE class in order to use different mutation strategies
class DE:
    """
    Class implementing the Differential Evolution (DE) algorithm in a
    generic manner, allowing the mutation strategy to be easily exchanged.
    """
    def __init__(self,
                 mutation_function,  # Mutation as a function
                 pop_size: int = 50,
                 num_iterations: int = 2000,
                 F: float = 0.8,  # typical value: 0.8
                 CR: float = 0.2):  # typical value: 0.2
        """
        Constructor of the DE algorithm.

        Args:
            mutation_function: Function implementing the mutation strategy.
            pop_size (int): Population size.
            num_iterations (int): Number of generations/iterations.
            F (float): Scaling factor for mutation.
            CR (float): Crossover rate (binomial crossover probability).
        """
        self.mutation_function = mutation_function
        self.pop_size = pop_size
        self.num_iterations = num_iterations
        self.F = F
        self.CR = CR

    def __call__(self, problem: ioh.problem.RealSingleObjective) -> None:
        dim = problem.meta_data.n_variables
        low_b, high_b = problem.bounds.lb[0], problem.bounds.ub[0]

        population = np.random.uniform(low_b, high_b, (self.pop_size, dim))
        fitness = np.array([problem(ind) for ind in population])

        for _ in range(self.num_iterations):
            for i in range(self.pop_size):
                # a. Mutation (now calling the external mutation function)
                mutant = self.mutation_function(
                    i=i,
                    population=population,
                    fitness=fitness,
                    F=self.F,
                    low_b=low_b,
                    high_b=high_b
                )

                # b. Crossover (Binomial) - unchanged
                trial_vector = np.copy(population[i])
                j_rand = np.random.randint(0, dim)
                crossover_mask = np.random.rand(dim) < self.CR
                crossover_mask[j_rand] = True
                trial_vector[crossover_mask] = mutant[crossover_mask]
                
                # c. Selection - unchanged
                trial_fitness = problem(trial_vector)
                
                if trial_fitness <= fitness[i]:
                    population[i] = trial_vector
                    fitness[i] = trial_fitness

In [3]:
# Mutation functions
# All comments and descriptions within this function were generated
# by the corresponding LLM model during the refinement stage.
def mutation_refined_gemini25flash(i: int, population: np.ndarray, fitness: np.ndarray, F: float,
    low_b: float, high_b: float) -> np.ndarray:
    """
    Implements an advanced hybrid mutation strategy that builds upon the
    winning strategy from Cycle 14. It preserves the guidance of
    `DE/current-to-pbest/1` and the dynamically blended difference term
    (`best-r1` and `r2-r3`), while introducing an adaptive `p` factor for
    selecting the `pbest` individual at each mutation.

    The base formula is:
    mutant = x_i + F_i * (x_pbest - x_i)
             + F_i * (alpha * (x_best - x_r1) + (1 - alpha) * (x_r2 - x_r3)).

    This strategy aims to achieve a robust balance:
    1. The first term (`x_pbest - x_i`) exploits information from a strong
       subset of the population. The novelty lies in selecting `pbest`
       from a dynamically chosen percentage `p` of the population
       (between 5% and 20% of the best individuals) for each mutation.
       This allows a finer adaptation between intensification (small `p`)
       and diversification (larger `p`), helping the strategy explore
       complex and multimodal fitness landscapes more flexibly, such as
       Gallagher21 or Weierstrass, where a fixed `p` could lead to premature
       stagnation or insufficient exploration.
    2. The second term is an adaptive combination (controlled by `alpha`)
       of an exploitation component (`x_best - x_r1`) and an exploration
       component (`x_r2 - x_r3`).

    The scaling factor F is adaptive, varying for each mutation (`F_i`) to
    dynamically adjust the magnitude of the difference vectors.
    """
    pop_size = len(population)
    
    # Generate a random F_i for this mutation, centered around the base F
    F_i = np.random.normal(loc=F, scale=0.1)
    F_i = max(0.1, F_i)  # Ensure F_i is positive for an effective mutation

    best_idx = np.argmin(fitness)

    # 1. Select p_best_idx: a random individual from the top P% of the population
    # KEY MODIFICATION: P is dynamically adjusted for each mutation
    p = np.random.uniform(0.05, 0.2)  # Percentage of top individuals to consider (e.g., 5% to 20%)
    
    # Obtain indices of individuals sorted by fitness (ascending for minimization)
    sorted_indices = np.argsort(fitness)
    
    # Determine how many pbest individuals to consider; ensure at least one
    num_pbest_candidates = max(1, int(pop_size * p))
    
    # Pool of the top P% individuals
    pbest_pool = list(sorted_indices[:num_pbest_candidates])
    
    # Ensure p_best_idx is different from 'i' so that (p_best - i) is meaningful
    pbest_candidates_for_diff = [idx for idx in pbest_pool if idx != i]
    
    if pbest_candidates_for_diff:
        p_best_idx = np.random.choice(pbest_candidates_for_diff)
    else:
        # Fallback if 'i' is the only candidate in the top P% or if the pool is empty
        # Select an individual from the population that is not 'i'
        non_i_indices = [idx for idx in range(pop_size) if idx != i]
        if non_i_indices:
            p_best_idx = np.random.choice(non_i_indices)
        else:
            # Extreme case: only one individual in the population (pop_size = 1)
            # This makes (population[p_best_idx] - population[i]) equal to zero
            p_best_idx = i

    # 2. Select r1, r2, r3: distinct random individuals
    # They must be different from 'i', 'best_idx', and 'p_best_idx', and mutually distinct

    all_pop_indices = list(range(pop_size))
    
    # Exclude indices already used in the formula or corresponding to the current individual (i)
    forbidden_r_indices = {i, best_idx, p_best_idx}
    
    selection_pool_for_r = [
        idx for idx in all_pop_indices if idx not in forbidden_r_indices
    ]

    # Robust handling for selecting three distinct indices (r1, r2, r3)
    if len(selection_pool_for_r) >= 3:
        r_indices = np.random.choice(selection_pool_for_r, 3, replace=False)
        r1, r2, r3 = r_indices[0], r_indices[1], r_indices[2]
    else:
        # Fallback for small populations or when many individuals are forbidden
        # Relax constraints, prioritizing indices different from 'i'
        fallback_pool_excluding_i = [idx for idx in all_pop_indices if idx != i]
        if len(fallback_pool_excluding_i) >= 3:
            r_indices = np.random.choice(fallback_pool_excluding_i, 3, replace=False)
            r1, r2, r3 = r_indices[0], r_indices[1], r_indices[2]
        else:
            # Extreme case: very small population (fewer than 4 individuals different from 'i')
            # or insufficient pool size. Replacement is allowed to avoid errors.
            r_indices = np.random.choice(all_pop_indices, 3, replace=True)
            r1, r2, r3 = r_indices[0], r_indices[1], r_indices[2]
            
    # 3. Generate the weighting factor alpha for the second difference term
    # Alpha is randomly selected between 0 and 1 for each mutation
    alpha = np.random.uniform(0, 1)
    
    # Compute the first difference term (current-to-pbest)
    term1_diff = population[p_best_idx] - population[i]
    
    # Compute the components of the second difference term
    term2_exploitation = population[best_idx] - population[r1]
    term2_exploration = population[r2] - population[r3]
    
    # blended_term2 = alpha * exploitation + (1 - alpha) * exploration
    blended_term2 = alpha * term2_exploitation + (1 - alpha) * term2_exploration

    # Apply the proposed hybrid mutation formula:
    # mutant = x_i + F_i * (x_pbest - x_i)
    #          + F_i * (alpha * (x_best - x_r1) + (1 - alpha) * (x_r2 - x_r3))
    mutant = population[i] + F_i * term1_diff + F_i * blended_term2
  
    return np.clip(mutant, low_b, high_b)
    

In [4]:
def analyze_file_list(root_folder, file_list):
    all_results = []

    print(
        f"Analyzing {len(file_list)} files from folder: "
        f"'{os.path.abspath(root_folder)}'"
    )

    for file_name in file_list:
        # Build the full path to the file
        file_path = os.path.join(root_folder, file_name)
        
        try:
            # Extract information from the file name
            match_name = re.match(r"IOHprofiler_f(\d+)_(.*)\.info", file_name)
            if not match_name:
                continue
            
            func_id = int(match_name.group(1))
            func_name = match_name.group(2)

            # Read the file content
            with open(file_path, 'r') as f:
                content = f.read()
            
            # Extract results from repeated runs
            lines = content.splitlines()
            results_line = ""
            for i, current_line in enumerate(lines):
                if current_line.strip() == '%':
                    if i + 1 < len(lines):
                        results_line = lines[i + 1]
                        break
            
            if not results_line:
                raise ValueError(
                    "Results line not found after '%' delimiter"
                )

            matches_data = re.findall(
                r'(\d+)\s*:\s*(\d+)\s*\|\s*([\d.eE+-]+)',
                results_line
            )
            if not matches_data:
                raise ValueError("No result entries were found.")

            # Convert extracted data to numerical lists
            best_fitness_list = [float(m[2]) for m in matches_data]
            evaluations_list = [int(m[1]) for m in matches_data]
            
            # --- COMPUTATIONS ---
            # Compute mean, standard deviation, and median
            avg_fitness = np.mean(best_fitness_list)
            std_fitness = np.std(best_fitness_list)
            median_fitness = np.median(best_fitness_list)
            avg_evals = np.mean(evaluations_list)

            # --- STORE RESULTS ---
            all_results.append({
                "Function_ID": func_id,
                "Function_Name": func_name,
                "Avg_Fitness": avg_fitness,
                "Std_Fitness": std_fitness,
                "Median_Fitness": median_fitness,
                "Avg_Evals": avg_evals
            })
        except FileNotFoundError:
            print(f"- Warning: File '{file_path}' not found")
        except Exception as e:
            print(f"- Error processing file {file_name}: {e}")
    
    if not all_results:
        return None
        
    return pd.DataFrame(all_results)


In [ ]:

de_algorithm = DE(mutation_function=mutation_refined_gemini25flash, pop_size=50, F=0.8, CR=0.2)

experiment = ioh.Experiment(
    algorithm=de_algorithm,
    fids=range(1, 25), #Function range
    iids=[1], #Instances
    dims=[5], #Dimensions
    reps=30, #Number of Repetitions
    zip_output=False,
    folder_name="mutation_refined_gemini25flash",#Name of the output folder where results are stored
    old_logger=True
)

# To run the experiment, simply call it
experiment.run()

In [ ]:
# Analyze and display the results
if __name__ == "__main__":
    ROOT_FOLDER = "mutation_refined_gemini25flash" 

    files_to_analyze = [
        "IOHprofiler_f1_Sphere.info",
        "IOHprofiler_f2_Ellipsoid.info",
        "IOHprofiler_f3_Rastrigin.info",
        "IOHprofiler_f4_BuecheRastrigin.info",
        "IOHprofiler_f5_LinearSlope.info",
        "IOHprofiler_f6_AttractiveSector.info",
        "IOHprofiler_f7_StepEllipsoid.info",
        "IOHprofiler_f8_Rosenbrock.info",
        "IOHprofiler_f9_RosenbrockRotated.info",
        "IOHprofiler_f10_EllipsoidRotated.info",
        "IOHprofiler_f11_Discus.info",
        "IOHprofiler_f12_BentCigar.info",
        "IOHprofiler_f13_SharpRidge.info",
        "IOHprofiler_f14_DifferentPowers.info",
        "IOHprofiler_f15_RastriginRotated.info",
        "IOHprofiler_f16_Weierstrass.info",
        "IOHprofiler_f17_Schaffers10.info",
        "IOHprofiler_f18_Schaffers1000.info",
        "IOHprofiler_f19_GriewankRosenbrock.info",
        "IOHprofiler_f20_Schwefel.info",
        "IOHprofiler_f21_Gallagher101.info",
        "IOHprofiler_f22_Gallagher21.info",
        "IOHprofiler_f23_Katsuura.info",
        "IOHprofiler_f24_LunacekBiRastrigin.info"
    ]

    final_summary_df = analyze_file_list(root_folder=ROOT_FOLDER, file_list=files_to_analyze)

    print(final_summary_df)

In [ ]:
# Save the results to a .csv file
output_csv = "mutation_refined_gemini25flash.csv"
final_summary_df.to_csv(output_csv, index=False, encoding="utf-8-sig")
print(f"CSV file saved to: {output_csv}")